# Langages de script – Python

## Visualisation vectorielle de corpus

### M2 Ingénierie Multilingue – INaLCO

clement.plancq@ens.fr

## Visualiser des données vectorielles
On a vu dans [un TP précédent](modele_vectoriel.ipynb) plusieurs façon de transformer de représenter des documents sous forme de vecteurs.

Une des applications de ce type de transformations est de visualiser des corpus pour pouvoir intuitivement des observations et des hypothèses sur leurs distributions.

## Votre pote `matplotlib`, là

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
X = np.random.rand(100)
Y = np.random.rand(100)
plt.scatter(X, Y)

In [ ]:
classes = np.random.randint(0, 2, 100)
plt.scatter(X, Y, c=classes)

## Voir en dimension 10000
Les représentations vectorielles de documents sont souvent en haute dimension
  - Quelques centaintes pour doc2vec et al.
  - Plusieurs millers pour les sacs de mots
  
On va donc devoir trouver des solutions pour se rammener à deux dimensions en gardant un maximum de pertinence

D'abord, un peu d'infrastructure

In [ ]:
pos = np.loadtxt('data/imdb/pos.tsv')
neg = np.loadtxt('data/imdb/neg.tsv')
display(pos.shape, neg.shape)

In [ ]:
X = np.concatenate([pos, neg])
X.shape

In [ ]:
cl = np.concatenate([np.zeros(pos.shape[0]), np.ones(neg.shape[0])])

On peut par exemple prendre les deux premières dimensions

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=cl, edgecolors='black')

C'est donc complètement nul. Et encore la première dimension a l'air un peu intéressante

### Trouver les dimensions intéressantes
On a voit que toutes les dimensions ne sont pas de la même nature
  - La plupart sont creuses : donc seuls quelques documents y ont une coordonée non-nulle
  - Pour quelques mots, on aura une bonne dispersion (ici pour *a*)

L'idée de l'Analyse en Composantes Principales c'est de trouver les dimensions qui expliquent le mieux la dispersion de l'ensemble du corpus.

(Par contre on s'autorise à combiner plusieurs dimension entre elles, ce qui revient à faire des rotations)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
Y = pca.fit_transform(X)
display(Y.shape)
plt.scatter(Y[:, 0], Y[:, 1], c=cl[:], edgecolors='k')

In [ ]:
plt.scatter(Y[:301, 0], Y[:301, 1], c=cl[:301], edgecolors='k')

In [ ]:
plt.scatter(Y[300:, 0], Y[300:, 1], c=cl[300:], edgecolors='k')

In [ ]:
pca3d = PCA(n_components=3)
Y3d = pca3d.fit_transform(X)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(Y3d[:, 0], Y3d[:, 1],  Y3d[:, 2], c=cl[:], edgecolors='k')

 ### t-SNE, la star
 T-distributed Stochastic Neighbor Embedding

 On part dans du lourdement non-interprétable : prend comme représentation en 2d du corpus celle dont la distribution spatiale ($\approx$ les distances entre documents est la plus proche de celle dans l'espace de départ)

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
Z = tsne.fit_transform(X)
plt.scatter(Z[:, 0], Z[:, 1], c=cl[:], edgecolors='k')

In [ ]:
tsne3 = TSNE(n_components=3)
Z3d = tsne3.fit_transform(X)

In [ ]:
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(Z3d[:, 0], Z3d[:, 1], Z3d[:, 2], c=cl[:], edgecolors='k')

Les auteurs conseillent de ne pas l'appliquer sur des dimensions trop élevées. Le must c'est donc d'enchaîner ACP et t-SNE

In [ ]:
pca = PCA(n_components=50)
Y50 = pca.fit_transform(X)
T = tsne.fit_transform(Y50)
plt.scatter(T[:, 0], T[:, 1], c=cl[:], edgecolors='k', alpha=0.5)